json -> image

In [6]:
import json
import os
import cv2
import numpy as np

DATA_DIR = r'C:\Users\swu\Desktop\Data\Preprocessing\realdata' # jpg, json 있는 경로
label_dir = os.path.join(DATA_DIR, 'label')

category_colors = {
    1: (255, 255, 0),  # 청록 -> single_jersey
    2: (0, 0, 255),  # 빨강 -> rib
    3: (0, 255, 0),  # 초록 -> purl
    4: (255, 0, 0),  # 파랑 -> ajour
    5: (0, 255, 255), # 노랑 -> moss
    0: (0, 0, 0)  # 검정색
}

def fill_unlabeled_areas_with(json_data, canvas_size):
    # 캔버스 생성 (검은색 배경: 0)
    image = np.zeros((canvas_size[1], canvas_size[0], 3), dtype=np.uint8)

    # 다각형 그리기
    for annotation in json_data["shapes"]:
        if annotation["label"] == "single_jersey":
            category_id = 1
        elif annotation["label"] == "rib":
            category_id = 2
        elif annotation["label"] == "purl":
            category_id = 3
        elif annotation["label"] == "ajour":
            category_id = 4
        elif annotation["label"] == "moss":
            category_id = 5
        else:
            continue  # 라벨이 없으면 skip
        
        polygon = annotation["points"]

        # 색상 선택
        color = category_colors.get(category_id, (255, 255, 255))  # 기본 색상은 흰색

        # 다각형 좌표를 픽셀 단위로 변환
        points = np.array([[int(x), int(y)] for x, y in polygon], dtype=np.int32)

        # 다각형 그리기
        cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)
        cv2.fillPoly(image, [points], color=color)

    # 이미지 저장
    output_path = os.path.join(label_dir, f"annotated_{json_data['imagePath']}")
    cv2.imwrite(output_path, image)

# 파일 읽기 및 처리
for filename in os.listdir(DATA_DIR):
    if filename.endswith('.json'):  # .json 파일만 처리
        input_file_path = os.path.join(DATA_DIR, filename)
        
        # 파일을 ascii 인코딩으로 열기
        with open(input_file_path, 'r', encoding='ascii') as file:
            json_data = json.load(file)
        
        canvas_size = (json_data["imageWidth"], json_data["imageHeight"])
        fill_unlabeled_areas_with(json_data, canvas_size)

label_list = os.listdir(label_dir)
print(f"Number of annotated labels: {len(label_list)}")


Number of annotated labels: 192


이미지 자르기

In [8]:
import os
import cv2
import json

# 파일 이름에 따라 자를 방향을 결정
top_cut_files = [
    10, 15, 16, 22, 28, 35, 37, 52, 59, 62, 63, 64, 74, 84, 88, 90,
    101, 107, 111, 120, 143, 148, 151, 157, 163, 166, 167, 168, 179,
    184, 186, 192, 200, 206, 219, 224, 230, 241, 242, 244, 268, 278
]
bottom_cut_files = [
    23, 26, 34, 43, 47, 49, 65, 73, 94, 93, 100, 109, 110, 112, 142,
    171, 172, 178, 181, 193, 195, 245, 251, 255, 275, 279, 293
]

# 이미지 폴더 경로 - 원본 이미지
image_dir = r"C:\Users\swu\Desktop\Data\Preprocessing\realdata"
crop_image_dir = r"C:\Users\swu\Desktop\Data\Preprocessing\crop_image"

# 크롭할 이미지 개수
DATA_NUM = 400 

# 크롭된 폴더가 없으면 생성
os.makedirs(crop_image_dir, exist_ok=True)

# 파일 이름 순서대로 반복
for i in range(1, DATA_NUM + 1):
    image_path = os.path.join(image_dir, f"{i}.jpg")

    # 이미지 파일이 없으면 건너뛰기
    if not os.path.exists(image_path):
        continue

    # 이미지 로드
    image = cv2.imread(image_path)

    if image is None:
        print(f"이미지를 찾을 수 없습니다: {image_path}")
        continue

    # 이미지 크기
    height, width, _ = image.shape

    # 정사각형 크기 결정 (가로와 세로 중 작은 값 선택)
    square_size = min(height, width)

    # 파일 이름에 따라 크롭 위치 결정
    if i in top_cut_files:
        start_x = (width - square_size) // 2
        start_y = 0
        end_x = start_x + square_size
        end_y = square_size
    elif i in bottom_cut_files:
        start_x = (width - square_size) // 2
        start_y = height - square_size
        end_x = start_x + square_size
        end_y = height
    else:
        start_x = (width - square_size) // 2
        start_y = (height - square_size) // 2
        end_x = start_x + square_size
        end_y = start_y + square_size

    # 이미지 크롭
    cropped_img = image[start_y:end_y, start_x:end_x]

    # 크롭된 이미지 저장
    output_image_name = f"{i}.jpg"
    output_image_path = os.path.join(crop_image_dir, output_image_name)
    cv2.imwrite(output_image_path, cropped_img)

    print(f"이미지 {i}: {output_image_name} 저장 완료")


이미지 6: 6.jpg 저장 완료
이미지 7: 7.jpg 저장 완료
이미지 8: 8.jpg 저장 완료
이미지 9: 9.jpg 저장 완료
이미지 10: 10.jpg 저장 완료
이미지 11: 11.jpg 저장 완료
이미지 14: 14.jpg 저장 완료
이미지 15: 15.jpg 저장 완료
이미지 16: 16.jpg 저장 완료
이미지 17: 17.jpg 저장 완료
이미지 18: 18.jpg 저장 완료
이미지 19: 19.jpg 저장 완료
이미지 20: 20.jpg 저장 완료
이미지 21: 21.jpg 저장 완료
이미지 22: 22.jpg 저장 완료
이미지 23: 23.jpg 저장 완료
이미지 24: 24.jpg 저장 완료
이미지 26: 26.jpg 저장 완료
이미지 27: 27.jpg 저장 완료
이미지 28: 28.jpg 저장 완료
이미지 29: 29.jpg 저장 완료
이미지 33: 33.jpg 저장 완료
이미지 34: 34.jpg 저장 완료
이미지 35: 35.jpg 저장 완료
이미지 37: 37.jpg 저장 완료
이미지 38: 38.jpg 저장 완료
이미지 39: 39.jpg 저장 완료
이미지 40: 40.jpg 저장 완료
이미지 41: 41.jpg 저장 완료
이미지 42: 42.jpg 저장 완료
이미지 43: 43.jpg 저장 완료
이미지 45: 45.jpg 저장 완료
이미지 46: 46.jpg 저장 완료
이미지 47: 47.jpg 저장 완료
이미지 48: 48.jpg 저장 완료
이미지 49: 49.jpg 저장 완료
이미지 50: 50.jpg 저장 완료
이미지 51: 51.jpg 저장 완료
이미지 52: 52.jpg 저장 완료
이미지 53: 53.jpg 저장 완료
이미지 59: 59.jpg 저장 완료
이미지 60: 60.jpg 저장 완료
이미지 61: 61.jpg 저장 완료
이미지 62: 62.jpg 저장 완료
이미지 63: 63.jpg 저장 완료
이미지 64: 64.jpg 저장 완료
이미지 65: 65.jpg 저장 완료
이미지 66: 66.jpg 저장 완료


In [9]:
import os
import cv2
import json

# 파일 이름에 따라 자를 방향을 결정
top_cut_files = [
    10, 15, 16, 22, 28, 35, 37, 52, 59, 62, 63, 64, 74, 84, 88, 90,
    101, 107, 111, 120, 143, 148, 151, 157, 163, 166, 167, 168, 179,
    184, 186, 192, 200, 206, 219, 224, 230, 241, 242, 244, 268, 278
]
bottom_cut_files = [
    23, 26, 34, 43, 47, 49, 65, 73, 94, 93, 100, 109, 110, 112, 142,
    171, 172, 178, 181, 193, 195, 245, 251, 255, 275, 279, 293
]

# 이미지 폴더 경로 - 라벨 이미지
image_dir = r"C:\Users\swu\Desktop\Data\Preprocessing\realdata\label"
crop_image_dir = r"C:\Users\swu\Desktop\Data\Preprocessing\crop_label"

# 크롭할 이미지 개수
DATA_NUM = 400 

# 크롭된 폴더가 없으면 생성
os.makedirs(crop_image_dir, exist_ok=True)

# 파일 이름 순서대로 반복
for i in range(1, DATA_NUM + 1):
    image_path = os.path.join(image_dir, f"annotated_{i}.jpg")

    # 이미지 파일이 없으면 건너뛰기
    if not os.path.exists(image_path):
        continue

    # 이미지 로드
    image = cv2.imread(image_path)

    if image is None:
        print(f"이미지를 찾을 수 없습니다: {image_path}")
        continue

    # 이미지 크기
    height, width, _ = image.shape

    # 정사각형 크기 결정 (가로와 세로 중 작은 값 선택)
    square_size = min(height, width)

    # 파일 이름에 따라 크롭 위치 결정
    if i in top_cut_files:
        start_x = (width - square_size) // 2
        start_y = 0
        end_x = start_x + square_size
        end_y = square_size
    elif i in bottom_cut_files:
        start_x = (width - square_size) // 2
        start_y = height - square_size
        end_x = start_x + square_size
        end_y = height
    else:
        start_x = (width - square_size) // 2
        start_y = (height - square_size) // 2
        end_x = start_x + square_size
        end_y = start_y + square_size

    # 이미지 크롭
    cropped_img = image[start_y:end_y, start_x:end_x]

    # 크롭된 이미지 저장
    output_image_name = f"annoatated_{i}.jpg"
    output_image_path = os.path.join(crop_image_dir, output_image_name)
    cv2.imwrite(output_image_path, cropped_img)

    print(f"이미지 {i}: {output_image_name} 저장 완료")


이미지 6: annoatated_6.jpg 저장 완료
이미지 7: annoatated_7.jpg 저장 완료
이미지 8: annoatated_8.jpg 저장 완료
이미지 9: annoatated_9.jpg 저장 완료
이미지 10: annoatated_10.jpg 저장 완료
이미지 11: annoatated_11.jpg 저장 완료
이미지 14: annoatated_14.jpg 저장 완료
이미지 15: annoatated_15.jpg 저장 완료
이미지 16: annoatated_16.jpg 저장 완료
이미지 17: annoatated_17.jpg 저장 완료
이미지 18: annoatated_18.jpg 저장 완료
이미지 19: annoatated_19.jpg 저장 완료
이미지 20: annoatated_20.jpg 저장 완료
이미지 21: annoatated_21.jpg 저장 완료
이미지 22: annoatated_22.jpg 저장 완료
이미지 23: annoatated_23.jpg 저장 완료
이미지 24: annoatated_24.jpg 저장 완료
이미지 26: annoatated_26.jpg 저장 완료
이미지 27: annoatated_27.jpg 저장 완료
이미지 28: annoatated_28.jpg 저장 완료
이미지 29: annoatated_29.jpg 저장 완료
이미지 33: annoatated_33.jpg 저장 완료
이미지 34: annoatated_34.jpg 저장 완료
이미지 35: annoatated_35.jpg 저장 완료
이미지 37: annoatated_37.jpg 저장 완료
이미지 38: annoatated_38.jpg 저장 완료
이미지 39: annoatated_39.jpg 저장 완료
이미지 40: annoatated_40.jpg 저장 완료
이미지 41: annoatated_41.jpg 저장 완료
이미지 42: annoatated_42.jpg 저장 완료
이미지 43: annoatated_43.jpg 저장 완료
이미지 45: annoatat

파일명 정렬

In [10]:
import os
import shutil

# 원본 폴더 경로
crop_image_dir = r"C:\Users\swu\Desktop\Data\Preprocessing\crop_image"
crop_label_dir = r"C:\Users\swu\Desktop\Data\Preprocessing\crop_label"

# 새로운 저장 폴더 경로
num_crop_image_dir = r"C:\Users\swu\Desktop\Data\Preprocessing\num_crop_image"
num_crop_label_dir = r"C:\Users\swu\Desktop\Data\Preprocessing\num_crop_label"

# 폴더가 없으면 생성
os.makedirs(num_crop_image_dir, exist_ok=True)
os.makedirs(num_crop_label_dir, exist_ok=True)

# 이미지 파일 및 라벨 파일 리스트 가져오기
image_files = sorted([f for f in os.listdir(crop_image_dir) if f.endswith('.jpg')])
label_files = sorted([f for f in os.listdir(crop_label_dir) if f.endswith('.jpg')])

# 이미지 파일 리네이밍 및 저장
for idx, image_file in enumerate(image_files, 1):
    # 새로운 파일 이름 생성
    new_image_name = f"{idx}.jpg"
    
    # 원본 이미지 경로
    old_image_path = os.path.join(crop_image_dir, image_file)
    # 새로운 이미지 경로
    new_image_path = os.path.join(num_crop_image_dir, new_image_name)
    
    # 이미지 파일 이동
    shutil.copy(old_image_path, new_image_path)
    print(f"'{image_file}' -> '{new_image_name}' (num_crop_image)")

# 라벨 파일 리네이밍 및 저장
for idx, label_file in enumerate(label_files, 1):
    # 새로운 파일 이름 생성
    new_label_name = f"annotated_{idx}.jpg"
    
    # 원본 라벨 경로
    old_label_path = os.path.join(crop_label_dir, label_file)
    # 새로운 라벨 경로
    new_label_path = os.path.join(num_crop_label_dir, new_label_name)
    
    # 라벨 파일 이동
    shutil.copy(old_label_path, new_label_path)
    print(f"'{label_file}' -> '{new_label_name}' (num_crop_label)")


'10.jpg' -> '1.jpg' (num_crop_image)
'100.jpg' -> '2.jpg' (num_crop_image)
'101.jpg' -> '3.jpg' (num_crop_image)
'102.jpg' -> '4.jpg' (num_crop_image)
'103.jpg' -> '5.jpg' (num_crop_image)
'104.jpg' -> '6.jpg' (num_crop_image)
'105.jpg' -> '7.jpg' (num_crop_image)
'106.jpg' -> '8.jpg' (num_crop_image)
'107.jpg' -> '9.jpg' (num_crop_image)
'109.jpg' -> '10.jpg' (num_crop_image)
'11.jpg' -> '11.jpg' (num_crop_image)
'110.jpg' -> '12.jpg' (num_crop_image)
'111.jpg' -> '13.jpg' (num_crop_image)
'112.jpg' -> '14.jpg' (num_crop_image)
'113.jpg' -> '15.jpg' (num_crop_image)
'114.jpg' -> '16.jpg' (num_crop_image)
'116.jpg' -> '17.jpg' (num_crop_image)
'118.jpg' -> '18.jpg' (num_crop_image)
'119.jpg' -> '19.jpg' (num_crop_image)
'120.jpg' -> '20.jpg' (num_crop_image)
'121.jpg' -> '21.jpg' (num_crop_image)
'122.jpg' -> '22.jpg' (num_crop_image)
'123.jpg' -> '23.jpg' (num_crop_image)
'124.jpg' -> '24.jpg' (num_crop_image)
'130.jpg' -> '25.jpg' (num_crop_image)
'132.jpg' -> '26.jpg' (num_crop_imag

데이터 증폭

In [13]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 경로 설정
data_path = r'C:\Users\swu\Desktop\Data\Preprocessing\rib'  # 데이터 경로
input_path = os.path.join(data_path, "num_crop_image")
label_path = os.path.join(data_path, "num_crop_label")
output_path_inputs = os.path.join(data_path, "augmented", "image")
output_path_labels = os.path.join(data_path, "augmented", "label")

# 출력 폴더 생성
os.makedirs(output_path_inputs, exist_ok=True)
os.makedirs(output_path_labels, exist_ok=True)

# 파일 로드
input_files = sorted([os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith('.jpg')])
label_files = sorted([os.path.join(label_path, f) for f in os.listdir(label_path) if f.endswith('.jpg')])

# 데이터 증폭 설정
data_gen_args = dict(
    rotation_range=360,         # 랜덤 회전 범위
    width_shift_range=0.2,      # 가로 이동 범위
    height_shift_range=0.2,     # 세로 이동 범위
    shear_range=0.2,            # 기울이기 범위
    horizontal_flip=True,       # 좌우 반전
    fill_mode='constant',       # 빈 부분 채우기
    cval=0                      # 채운 값 (기본값은 0)
)

# ImageDataGenerator 생성
image_datagen = ImageDataGenerator(**data_gen_args)
label_datagen = ImageDataGenerator(**data_gen_args)

# 랜덤 색 변환 함수
def random_color_transform(image, hue_shift, sat_shift, val_shift):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  # BGR -> HSV
    hsv_image[..., 0] = (hsv_image[..., 0] + hue_shift) % 180  # Hue 변경
    hsv_image[..., 1] = np.clip(hsv_image[..., 1] * sat_shift, 0, 255)  # Saturation 변경
    hsv_image[..., 2] = np.clip(hsv_image[..., 2] * val_shift, 0, 255)  # Brightness 변경
    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)  # HSV -> BGR

# 증폭 시작
num_augmentations_per_image = 10  # 각 이미지당 생성할 데이터 수

for idx, (input_file, label_file) in enumerate(zip(input_files, label_files)):
    input_img = cv2.imread(input_file)
    label_img = cv2.imread(label_file)

    input_img = np.expand_dims(input_img, 0)  # 배치 차원 추가
    label_img = np.expand_dims(label_img, 0)  # 배치 차원 추가

    input_gen = image_datagen.flow(input_img, batch_size=1, seed=42)
    label_gen = label_datagen.flow(label_img, batch_size=1, seed=42)

    for aug_idx in range(num_augmentations_per_image):
        aug_input = input_gen.next()[0].astype(np.uint8)
        aug_label = label_gen.next()[0].astype(np.uint8)

        # 색 변환
        hue_shift = np.random.randint(0, 180)
        sat_shift = np.random.uniform(0.5, 1.5)
        val_shift = np.random.uniform(0.5, 1.5)

        aug_input = random_color_transform(aug_input, hue_shift, sat_shift, val_shift)

        # 이미지 파일 번호 설정
        img_num = idx * 10 + aug_idx + 193  # 시작 번호 설정

        # 파일 경로 설정
        input_output_file = os.path.join(output_path_inputs, f'{img_num}.jpg')
        label_output_file = os.path.join(output_path_labels, f'annotated_{img_num}.jpg')

        # 이미지 저장
        cv2.imwrite(input_output_file, aug_input)
        cv2.imwrite(label_output_file, aug_label)

print(f"총 {len(input_files) * num_augmentations_per_image}개의 데이터 증폭 완료")


총 1920개의 데이터 증폭 완료
